## Forward Modeling

The goal of this notebook is to "illustrate how users might do forward-model simulations of a dispersed 2-D grism spectrum from a one-D model input spectrum." This is currently possible at least with the Grizli package, SBE, PyLINEAR, NIRCAN_Gsim and presumably aXeSIM, although word on the street is that there is currently only one individual who knows how to/has gotten aXeSIM working.

#### Pseudo code

An easy 2D spectrum modeling process for the user might look something like this:
```
  import astrogrism as ag
  from specutils import Spectrum1D
  
  target = [direct image of object]
  # Load a configuration file containing information about the grism
  grism_config = ag.load_config(filename/instrument)
  # Load a 1D spectrum for the object
  spec1d = Spectrum1D.read(spectrum file)
  # Model the dispersed 2D spectrum of the object
  spec2d = ag.disperse_object(target, grism_config, spectrum_1d=spec1d)
```

A full image could be dispersed by providing a segmentation map and a dictionary of spectra that apply to each object. On the backend this would farm out the dispersion of each object to the ag.disperse_object function via multiprocessing (I'm imagining that the individual object dispersions would be vectorized rather than multiprocessed):
```
  image_file = "path_to_flt"
  seg_file = "path_to_segmentation_file"
  dispersed_image = ag.disperse_all_objects(image_file, seg_file, grism_conf, spectrum_files)
```

### Existing Packages

#### Grizli

Grizli contains a `GrismDisperser` class in its `model` module that handles doing forward transformations. This object takes a grismconf files and stores as attributes all of the information needed to calculate the dispersion. It also can take as input a direct image and segmentation map (as well as an ID corresponding to the object of interest in the segmentation map). The user can then use `GrismDisperser.compute_model()` to generate a model 2D spectrum given (all optional) a segment ID, thumbnail direct image, and 1D spectrum. If no 1D spectrum is given a flat spectrum is assumed. 

The actual dispersion calculation, once `compute_model` does all the prep, is done by `grizli.utils_c.disperse.disperse_grism_object`, which appears to be Cython code (I'm unfamiliar with Cython).

Ref: https://grizli.readthedocs.io/en/master/api/grizli.model.GrismDisperser.html and especially https://grizli.readthedocs.io/en/master/_modules/grizli/model.html#GrismDisperser.compute_model

#### PyLINEAR

PyLINEAR uses parallelization via a `multiprocessing.Pool`, which is called from the `simulate` function to run multiple `simulateWorker`s, each of which handles the actual source dispersion and output file creation. The input to the `simulate` function is a `conf`, a dictionary that contains parameters for the simulation run, and `sources`, a list of sources corresponding to the source IDs in a file defined in `conf['sedlst']`. Note that the `conf` input to the `simulate` function is NOT the grismconf file, rather the grismconf file to be used is defined in the `conf` input.

This definitely seems geared toward pipelining/doing many sources, and isn't really modularized such that one could easily use the functionality for a quick simulation of one source. Also in that vein, it seems to require a lot of prep for a run (lots of configurations to set up, source list files, etc), although much of that may be automated elsewhere in the code or available as presets/files from the relevant mission.

Ref: https://github.com/Russell-Ryan/pyLINEAR/blob/master/pylinear/modules/simulation/simulation.py

#### NIRCAM_Gsim

The NIRCAM_Gsim methodology is similar to that of Grizli, however the class used to for dispersion simulation is tied to a specific observation (a single FLT file + config) rather than a grism definition. Similar to PyLINEAR, `multiprocessing.Pool` is used in various places for parallelization of dispersing each individual pixel. It's easier than in PyLINEAR to get at the underlying forward modeling code independent of the `observation` object that is built on an entire FLT file, since the dispersion code is split out into a separate function `disperse.dispersed_pixel(x0s, y0s, f, order, C, ID, extrapolate_SED)`, although generating the input to that function (specifically the pixel lists) is less intuitive than just inputting an object postage stamp.

Ref: https://github.com/npirzkal/NIRCAM_Gsim/blob/master/NIRCAM_Gsim/observations/observations.py and https://github.com/npirzkal/NIRCAM_Gsim/blob/master/NIRCAM_Gsim/disperse/disperse.py

### Moving forward: AstroGrism modeling principles

#### AstroGrism Potential Improvements

So far, I'm under the impression that the AstroGrism project can potentially improve the forward modeling part of grism analysis in a few areas (fairly applicable to the overall grism picture):

1. Existing packages largely seem to take an "everything and the kitchen sink" approach. Trimming to a lighter, more focused set of capabilities and ditching many large underlying code bases that are only used in one or two places would make it cheaper and easier to get at specific functionality like forward modeling.
2. Following from two, in some existing packages the forward modeling is bundled up in pipelining/batch processing and hard to get at for use in individual instances. We can properly modularize the code to better expose underlying functions. This also helps improve ease of maintenance (see 3).
3. Trimming dependencies and refactoring would likely make the code easier to maintain than existing packages. Grizli for example has underlying Cython code that may be able to be ditched without severe performance hits.
4. Performance (maybe!): looking at the existing source code I've noticed a lot of loops, which automatically makes my brain yell "vectorize!". Of course the author's of the existing packages may have considered/tried this and decided against it for good reason, but it isn't clear to me that NIRCAM_Gsim's strategy of dispersing each pixel separately, but using a `multiprocessing.Pool`, is faster than might be achieved with some clever vectorization.

#### AstroGrism forward modeling outline


